# Proceso GLUE Eventos Amplitude

## 1. Cargamos las librerias

In [1]:
!pip install -q awswrangler

In [2]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import dask.array as da
import boto3
import ast
from datetime import datetime, timedelta
import awswrangler as wr
from itertools import chain
import gc
import sys
import time
from sklearn import preprocessing

glue = boto3.client('glue')
s3 = boto3.resource('s3')
ssm = boto3.client('ssm') 
lakeformation = boto3.client('lakeformation')

### 2. Armamos el proceso en "GLUE LTV-RFM AR"

In [72]:
%%writefile get_data_amplitude.py

import sys
import pyspark.sql.functions as func
from awsglue.dynamicframe import DynamicFrame
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.sql.types import *
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
import pyspark.sql.functions as F
import json
import boto3
import ast
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import gc
import sys
from pyspark.conf import SparkConf
import pandas as pd

print('Lectura de parámetros')

# ----------------------------------------------------------------------------------
print('NOW:', datetime.now())

args = getResolvedOptions(sys.argv,
                          ['bucket_amplitude_data', 
                           'today', 
                           'kms_key_arn', 
                           'recommendations_bucket'])

bucket_amplitude_data = args['bucket_amplitude_data']
recommendations_bucket = args['recommendations_bucket']
kms_key_id = args['kms_key_arn']
today = args['today']


#--------------------------------------------------------------------------------------------------------------

print('Spark Configuración')

spark_conf = SparkConf().setAll([
  ("spark.hadoop.fs.s3.enableServerSideEncryption", "true"),
  ("spark.hadoop.fs.s3.serverSideEncryption.kms.keyId", kms_key_id)
])

sc = SparkContext(conf=spark_conf)
glueContext = GlueContext(sc)
spark = glueContext.spark_session
logger = glueContext.get_logger()



print('Crear objetos S3-ssm')
# ----------------------------------------------------------------------------------
s3 = boto3.resource('s3')
s3_client = boto3.client('s3')
ssm = boto3.client('ssm')

#--------------------------------------------------------------------------------------------------------------
print('Parámetros:')
path_key_amplitude = 'ar/amplitude/tb_ar_amplitude_events_stage/'

## FECHAS INTERVALO
#print('1. CALCULO DE FECHAS')
##Today llevado al primero del mes menos 1 día
#today = datetime.strptime(today, '%Y-%m-%d').date().replace(day=1)
#last_day=(today-pd.offsets.DateOffset(days=1)).date()
##
#first_day=(last_day-pd.offsets.DateOffset(days=365)).date()
#
#print('2. Intevalo de fechas analizada: ',first_day,'y',last_day)

def first_and_last(today):
    fecha=datetime.strptime(today, '%Y-%m-%d').date()
    first_day=fecha.replace(day=1)
    next_month = fecha.replace(day=28) + timedelta(days=4)
    last_day_of_month = next_month - timedelta(days=next_month.day)
    return first_day,last_day_of_month

print('Declaración de funciones')
def list_objects_function(buckets_, first_day, last_day, keys_, retrieve_last=False):
    
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(buckets_)
    files_in_bucket = list(bucket.objects.all())
    files_objets = [f"s3://{buckets_}/" + i.key for i in files_in_bucket if
                        (i.key.find(keys_) >= 0) and (i.key.find('.parquet') >= 0)]
    df_bucket_files = pd.DataFrame({
            'key': [i[:(i.find('dt=') + 14)] for i in files_objets],
            'path': files_objets,
            'date': pd.to_datetime([i[(i.find('dt=') + 3):(i.find('dt=') + 13)] for i in files_objets])
        })
    files=list(df_bucket_files.loc[df_bucket_files['date'].between(str(first_day),str(last_day)),'path'].values)
    return files



map_events = {
    "cuoti_selecciona_elegircuotas" : "cuotificaciones",
    "cuoti_sigue_seleccion_consumos" : "cuotificaciones",
    "prestamos_selecciona_simular_prestamo": "prestamos",
    "prestamos_espera": "prestamos",
    "general_ingresa_promociones": "promociones",
    "recargas_click_empezar": "recargas",
    "recargas_click_repetir": "recargas",
    "transferencia_selecciona_tieneuala": "transferencia_c2c",
    "transferencia_selecciona_notieneuala": "transferencia_cvu",
    "general_ingresa_cobros": "cobros",
    "cobros_acepta_tyc" : "cobros",
    "cobros_elige_link": "cobros",
    "cobros_elige_mpos": "cobros",
    "pagos_empezar": "pago_servicios",
    "click_inversiones":"inversiones"
}

eventos_recommendations = list(map_events.keys())
#-----------------------------------------------------------------------------------------------------------------
first_day,last_day = first_and_last(today)
print('Primer dia',first_day)
print('Ultimo dia',last_day)

files_objets_amplitude = list_objects_function(bucket_amplitude_data, first_day, last_day ,path_key_amplitude)

print(f'Hay {len(files_objets_amplitude)} archivos de survival en la carpeta')
df_amplitude = spark.read.parquet(*files_objets_amplitude).select(['user_id',"os_name","event_type","event_time"])
df_amplitude=df_amplitude.filter(df_amplitude.event_type.isin(eventos_recommendations))

df_amplitude = df_amplitude.withColumn('year_month', F.date_format(df_amplitude.event_time,'YYYY-MM'))

df_amplitude = df_amplitude.drop("event_time")

df_amplitude = df_amplitude.na.replace(map_events,1,"event_type")

df_amplitude = (df_amplitude    
      .groupBy(['user_id', 'event_type', 'year_month'])
      .agg(F.count('event_type').alias('cant'),
           F.max('os_name').alias('os_name'))
      .groupBy(['user_id','year_month','os_name'])
      .pivot("event_type")
      .agg(F.sum('cant'))
      .na.fill(0)
      )

df_amplitude.write\
     .format('parquet')\
     .save(f's3://{recommendations_bucket}/data/raw/amplitude/dt={str(first_day)}', mode='overwrite')

print('Ubicación files', f's3://{recommendations_bucket}/data/raw/amplitude/dt={str(first_day)}')

#DELETE $FOLDER$

def retrieve_files(path, file_type, list_dates):
    bucket=path.split('/')[2]
    prefix='/'.join(path.split('/')[3:])
    list_objects=list(s3.Bucket(bucket).objects.all())
    list_objects=[f's3://{bucket}/{i.key}' for i in list_objects if ((i.key.find(prefix)>=0) & any(x in i.key.lower() for x in list_dates) & (i.key.find(file_type)>=0))]
    return list_objects


delete_files = retrieve_files(path=f's3://{recommendations_bucket}/data/', file_type='$folder$', list_dates=['$folder$'])
print('Files to delete', delete_files)
files_keys=[]
for i in range(0,len(delete_files)):
    files_keys=files_keys+[{'Key':('/').join(delete_files[i].split('/')[3:])}]
if len(files_keys)>0:
    s3_client.delete_objects(Bucket=recommendations_bucket,
                             Delete={'Objects':files_keys})
del delete_files
gc.collect()

print(df_amplitude.show())
print((df_amplitude.count(), len(df_amplitude.columns)))

Overwriting get_data_amplitude.py


In [73]:
job_name='test-job_recommendations_amplitude'

In [74]:
# borrar job
#glue.delete_job(
#    JobName=job_name
#)

## 3. Generamos los parametros

In [75]:
today = '2021-01-10'
bucket_amplitude_data='churn-ds-stage'  ## AFIP, GP, etc
recommendations_bucket='test-uala-arg-datalake-aiml-recommendations'  # Para outputs
kms_key_arn='arn:aws:kms:us-east-1:322149183112:key/9cc44b23-c5e9-46cb-9987-0982d21f8d00' ## key para desencriptar

In [76]:
s3 = boto3.resource('s3')

# Guardar el archivo .py
s3.meta.client.upload_file('get_data_amplitude.py', 
                           recommendations_bucket, #bucket
                           'artifacts/code/amplitude/get_data_amplitude.py' #key+filename
)
print('.py uploaded')

.py uploaded


## 4. Creamos el job de GLUE

In [77]:
#job = glue.create_job(Name=job_name, 
#                      GlueVersion='2.0',
#                      Role='ML_AWSGlueServiceRole',
#                      Command={'Name': 'glueetl',
#                               'ScriptLocation': f's3://{recommendations_bucket}/artifacts/code/amplitude/get_data_amplitude.py'},
#                      DefaultArguments={
#                        '--additional-python-modules': 'dateutil==2.8.1'},
#                      MaxCapacity=3
#                      )

In [78]:
job_run = glue.start_job_run(
    JobName = job_name,
    Arguments = {
        '--today':today,
        '--bucket_amplitude_data': bucket_amplitude_data,
        '--recommendations_bucket': recommendations_bucket,
        '--kms_key_arn': kms_key_arn
    } 
)

In [79]:
print(job_run)

{'JobRunId': 'jr_cf2978c1ed6c989cac356896b755adc916606f4449ff8bf16a36c86932e7beca', 'ResponseMetadata': {'RequestId': 'ded488e5-e827-4452-bb2d-5e053957841f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 20 May 2021 19:56:31 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '82', 'connection': 'keep-alive', 'x-amzn-requestid': 'ded488e5-e827-4452-bb2d-5e053957841f'}, 'RetryAttempts': 0}}


In [80]:
MAX_WAIT_TIME=time.time() + 60*10 # 1 hour
max_time = time.time() + MAX_WAIT_TIME
while time.time() < max_time:
    response=glue.get_job_run(JobName=job_name, RunId=job_run['JobRunId'])
    status = response['JobRun']['JobRunState']
    print('Job run: {}'.format(status))
    
    if status == 'SUCCEEDED' or status == 'FAILED':
        break
        
    time.sleep(45)

Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: SUCCEEDED


## 5. Controlamos la carga de datos en el bucket


In [60]:
df=wr.s3.read_parquet(f's3://test-uala-arg-datalake-aiml-recommendations/data/raw/amplitude',dataset=True,partition_filter=lambda x: '2021-01-01' in x["dt"])

In [62]:
df.head(5)

,user_id,year_month,os_name,cobros,cuotificaciones,inversiones,pago_servicios,prestamos,promociones,recargas,transferencia_c2c,transferencia_cvu,dt
0,00360d0c-2638-458b-a1b6-144cb3ba4b78,2021-01,ios,0,0,4,0,0,0,0,4,1,2021-01-01
1,00509604-ce55-47db-93f6-6bca3867ef05,2021-01,android,0,0,0,0,0,0,0,10,4,2021-01-01
2,005e8359-5408-4b28-b2a0-a5ef6955f158,2021-01,android,0,0,0,0,0,0,0,2,1,2021-01-01
3,0072cfda-2213-4b1a-b4bf-3969ccacde3f,2021-01,android,2,0,0,0,0,0,4,4,1,2021-01-01
4,0074aa89-9148-4d94-9bee-e55bd591d038,2021-01,android,0,2,13,2,0,0,5,10,4,2021-01-01


## 6. Carga histórica


In [15]:
bucket_amplitude_data='churn-ds-stage'  ## AFIP, GP, etc
recommendations_bucket='test-uala-arg-datalake-aiml-recommendations'  # Para outputs
kms_key_arn='arn:aws:kms:us-east-1:322149183112:key/9cc44b23-c5e9-46cb-9987-0982d21f8d00' ## key para desencriptar
list_fechas=['2020-05-01','2020-06-01','2020-07-01','2020-08-01','2020-09-01','2020-10-01','2020-11-01','2020-12-01',
            '2021-01-01','2021-02-01','2021-03-01']
job_name='test-job_recommendations_amplitude'

In [18]:
#for value in list_fechas:
    print("Procesando:",value[:7])
    job_run = glue.start_job_run(
        JobName = job_name,
        Arguments = {
            '--today':value,
            '--bucket_amplitude_data': bucket_amplitude_data,
            '--recommendations_bucket': recommendations_bucket,
            '--kms_key_arn': kms_key_arn
        } 
    )
    MAX_WAIT_TIME= 3600 # 1 hour
    max_time = time.time() + MAX_WAIT_TIME
    while time.time() < max_time:
        response=glue.get_job_run(JobName=job_name, RunId=job_run['JobRunId'])
        status = response['JobRun']['JobRunState']
        print('Job run: {}'.format(status))

        if status == 'SUCCEEDED':
            time.sleep(60)
            break
        elif status == 'FAILED':
            print ("Error para fecha:",value[:7]," \n" )
            sys.exit(1)
        time.sleep(45)

Procesando: 2020-08
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: SUCCEEDED
Procesando: 2020-09
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: SUCCEEDED
Procesando: 2020-10
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: SUCCEEDED
Procesando: 2020-11
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: SUCCEEDED
Procesando: 2020-12
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: SUCCEEDED
Procesando: 2021-01
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: SUCCEEDED
Procesando: 2021-02
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNIN